In [5]:
import pandas as pd
import numpy as np
import cv2
import os
import tifffile
from scipy.ndimage import binary_closing, generate_binary_structure
from scipy.spatial.distance import euclidean


In [9]:
def readfile(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Get the maximum x, y, and z values to determine the shape of the output array
    max_x = int(df['x_in_pix'].max())
    max_y = int(df['y_in_pix'].max())
    max_z = int(df['z_in_pix'].max())

    # Create a new NumPy array with the desired shape, initialized with zeros
    output_array = np.zeros((max_x + 1, max_y + 1, max_z + 1)).astype('int16')

    # Iterate through the DataFrame and store the cell index at the corresponding x, y, z position in the output array
    for index, row in df.iterrows():
        x, y, z = int(row['x_in_pix']), int(row['y_in_pix']), int(row['z_in_pix'])
        cell_index = int(row['Cell Index'])
        output_array[x, y, z] = cell_index

    # Add a padding of 1 layer of zeros around the existing array
    padded_array = np.pad(output_array, pad_width=10, mode='constant', constant_values=0)
    
    return padded_array


def closing_by_scipy(labels):
    # Create an empty array to store the processed labels
    closing_labels = np.zeros_like(labels).astype('int16')

    # Get the unique labels in the labeled matrix
    unique_labels = np.unique(labels)

    # Define the structure element for closing (you can adjust the connectivity parameter)
    # structure_element = generate_binary_structure(rank=3, connectivity=2)
    size = (100, 100, 100)
    radius = 10
    center = np.array(size) / 2
    structure_element = np.zeros(size, dtype=bool)

    for index in np.ndindex(structure_element.shape):
        if euclidean(center, index) <= radius:
            structure_element[index] = True
    
    # Loop through each unique label
    for label in unique_labels:
        if label == 0:  # Skip the background label
            continue

        # Create a binary mask for the current label
        mask = (labels == label).astype('int8')
        
        # Perform closing on the binary image
        closed_mask = binary_closing(mask, structure=structure_element)
        
        # Replace the original values in the processed labels with the updated values from the eroded mask
        closing_labels[closed_mask == 1] = label
    
    return closing_labels

In [ ]:
os.chdir('C:/Users/14187/HuangLab/spatial_transcriptome/2_segmentation_show/')

In [4]:
cell_array = readfile(csv_file = './rna_labeled.csv')

In [11]:
sub_cell_array = cell_array[1000:1300, 1000:1300, :]

In [12]:
closed_image = closing_by_scipy(sub_cell_array)
# Save the closed image as a new 3D TIFF
tifffile.imwrite('./output.tif', np.moveaxis(closed_image, 2, 0))